# Words We Like

In [1]:
!pip3 install openai

In [ ]:
import os
from typing import List, Dict, Optional

import openai

In [2]:
OPENAI_KEY_ENV_VAR = 'OPENAI_API_KEY'
OPENAI_USER = 'Employee X'

HOME_LABEL = 'The Home of the Family X in rural Manitoba, comprising three young kids, one father and one mother'

In [ ]:
openai.api_key = os.environ.get(OPENAI_KEY_ENV_VAR)

In [ ]:
class OpenAIChatCompletionError(Exception):
    pass

def send_request_to_openai_chat_completion(
        messages: List[Dict[str, str]],
        model: str = 'gpt-3.5-turbo',
        functions: Optional[List[str]] = None,
        function_call: str = 'auto',
        temperature: float = 1.0,
        max_tokens: int = 256,
        top_p: float = 1.0,
        frequency_penalty: float = 0.0,
        presence_penalty: float = 0.0,
        stop: Optional[List] = None,
        user: Optional[str] = None,
        n_completions: int = 1
) -> OpenAIObject:
    '''Function that converts calling arguments into a request to the OpenAI chat completion API.
    Includes validation and exception handling

    See https://platform.openai.com/docs/api-reference/chat/create

    '''
    for message in messages:
        for key, value in message.items():
            if not key in ['role', 'content', 'name', 'function_call']:
                raise ValueError('Invalid key in message: {}'.format(key))
            if key == 'role' and not value in ['system', 'user', 'assistant', 'function']:
                raise ValueError('Invalid role in message: {}'.format(value))

    kwargs = {
        'model': model,
        'messages': messages,
        'functions': functions,
        'function_call': function_call,
        'temperature': temperature,
        'max_tokens': max_tokens,
        'top_p': top_p,
        'frequency_penalty': frequency_penalty,
        'presence_penalty': presence_penalty,
        'stop': stop,
        'user': user,
        'n': n_completions
    }
    if functions is None:
        del kwargs['functions']
        del kwargs['function_call']
    if stop is None:
        del kwargs['stop']
    if user is None:
        del kwargs['user']

    try:
        openai_completion = openai.ChatCompletion.create(**kwargs)
    except InvalidRequestError as e:
        raise OpenAIChatCompletionError(
            'Invalid request to OpenAI API. Check model parameters. The error message was: {}'.format(e))
    except AuthenticationError as e:
        raise OpenAIChatCompletionError(
            'Authentication error. Check your API key. The error message was: {}'.format(e))
    except PermissionError as e:
        raise OpenAIChatCompletionError(
            'Request exceed permissions. Check your API settings. The error message was: {}'.format(e))
    except APIConnectionError as e:
        raise OpenAIChatCompletionError(
            'Unable to connect to API. Check your connection and API settings. The error message was: {}'.format(e))
    except RateLimitError as e:
        raise OpenAIChatCompletionError(
            'Rate limit exceeded. The error message was: {}'.format(e))
    except Exception as e:
        raise OpenAIChatCompletionError(
            'General OpenAI call error. The error message was: {}'.format(e))

    return openai_completion


In [ ]:
def get_sensor_state() -> Dict:
    return {
        'temperature indoor, sensor 1' : faker.sensors.get('temperature indoor')
    }

In [ ]:
def get_actuator_state() -> Dict:
    return {
        'thermostat 1' : faker.actuators.get('thermostat 1'),
        'luminaire, bedroom' : faker.actuators.get('luminaire 1')
    }

In [ ]:
def set_actuator_state(**kwargs) -> bool:
    faker.actuators.set(**kwargs)
    return True

In [ ]:
def translate_state() -> str:
    description = 'At the present time, {} is in the following state:\n'.format(HOME_LABEL)
    for key, value in get_sensor_state():
        description += '* The {} is at {}{}\n'.format(key, value.x, value.unit)
        
    description = 'Furthermore, at the present time, actuators are set to:\n'
    for key, value in get_actuator_state():
        description += '* The {} is at {}{}\n'.format(key, value.x, value.unit)
        
    return description